In [4]:
import os 

In [1]:
%pwd 

'c:\\Users\\ahmed\\OneDrive\\Documents\\Documents\\coding\\data science\\DataScience\\MLOPS-Project\\research'

I want to move to my project folder 

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\ahmed\\OneDrive\\Documents\\Documents\\coding\\data science\\DataScience\\MLOPS-Project'